In [1]:
import sys
sys.path.append("/Users/aaron/phd/impact-of-contamination-on-taxonomy")

In [2]:
from workflow.fastani_contig_split.d_report_results import FastAniContigSplitReportResultsFastAni
from collections import Counter

In [3]:
DF_RESULTS = FastAniContigSplitReportResultsFastAni().output().read_cached()


# Subset the results to bacteria only
DF_RESULTS = DF_RESULTS[DF_RESULTS['expected_tax'].apply(lambda x: x.startswith('d__Bacteria'))]

DF_RESULTS.head()

[17/01/2023 15:20:01] - 15.2MiB = /srv/home/uqamussi/projects/gunc-chimeras/output/fastani_contig_split/results_report_k16_s5000__p50.tsv -> /tmp/gunc-cache/srv/home/uqamussi/projects/gunc-chimeras/output/fastani_contig_split/results_report_k16_s5000__p50.tsv


,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
0,GCA_000143435.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,s,s,GCF_001435955.1,GCF_001435955.1,97.3962,97.3042,0.8174,0.7677,sp_rep,sp_rep,True,True,True
1,GCA_000153745.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True
2,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_003697165.2,NaN,96.5879,0.0000,0.7680,0.0000,sp_rep,no_af,True,False,False
3,GCA_000161815.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,s,s,GCF_001681715.1,GCF_001681715.1,97.1446,96.9094,0.9309,0.8098,sp_rep,sp_rep,True,True,True
4,GCA_000166735.2,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True


In [12]:
DF_RESULTS[DF_RESULTS['gid'] == 'GCA_016203935.1']

,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
7282,GCA_016203935.1,d__Bacteria;p__Patescibacteria;c__Andersenbact...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,no_af,no_af,True,True,True


In [4]:
# Report how many of the keep halves are the same as R207
N_KEEP_SAME_R207 = len(DF_RESULTS[DF_RESULTS['keep_same_as_207'] == True])
N_KEEP_DIFF_R207 = len(DF_RESULTS[DF_RESULTS['keep_same_as_207'] == False])
N_KEEP_DIFF_R207_BUT_DISC_SAME =  len(DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == True)])
print(f'There are {N_KEEP_SAME_R207:,} ({N_KEEP_SAME_R207/len(DF_RESULTS):.2%}) keep halves that are identical to the expected R207 taxonomy.')
print(f'There are {N_KEEP_DIFF_R207:,} ({N_KEEP_DIFF_R207/len(DF_RESULTS):.2%}) keep halves that are different to the expected R207 taxonomy.')
print(f'Of those, there are {N_KEEP_DIFF_R207_BUT_DISC_SAME:,} where the chimeric half has the expected R207 taxonomy (we picked the wrong half?)')
print('-' * 80)

N_DISC_SAME_R207 = len(DF_RESULTS[DF_RESULTS['disc_same_as_207'] == True])
N_DISC_DIFF_R207 = len(DF_RESULTS[DF_RESULTS['disc_same_as_207'] == False])
print(f'There are {N_DISC_SAME_R207:,} ({N_DISC_SAME_R207/len(DF_RESULTS):.2%}) chimeric halves that are identical to the expected R207 taxonomy.')
print(f'There are {N_DISC_DIFF_R207:,} ({N_DISC_DIFF_R207/len(DF_RESULTS):.2%}) chimeric halves that are different to the expected R207 taxonomy.')

print('-' * 80)

N_EITHER_DIFF = len(DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) | (DF_RESULTS['disc_same_as_207'] == False)])
print(f'There are {N_EITHER_DIFF:,} ({N_EITHER_DIFF/len(DF_RESULTS):.2%}) cases were either half has an incorrect taxonomic assignment')

print('-' * 80)

N_ONE_RIGHT_ONE_WRONG = len(DF_RESULTS[
    ((DF_RESULTS['keep_same_as_207'] == True) & (DF_RESULTS['disc_same_as_207'] == False)) |
     ((DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == True))
])
print(f'There are {N_ONE_RIGHT_ONE_WRONG:,} ({N_ONE_RIGHT_ONE_WRONG/len(DF_RESULTS):.2%}) cases where one half was wrong, and the other half was correct.')

N_NEITHER_CORRECT = len(DF_RESULTS[(DF_RESULTS['disc_same_as_207'] == False) & (DF_RESULTS['keep_same_as_207'] == False)])
print(f'There are {N_NEITHER_CORRECT:,} cases where neither half is correct.')

print('-' * 80)

N_KEEP_EQ_DISC = len(DF_RESULTS[DF_RESULTS['keep_same_as_disc'] == True])
print(f'There are {N_KEEP_EQ_DISC:,} ({N_KEEP_EQ_DISC/len(DF_RESULTS):.2%}) halves that are identical in assignment to each other.')


print('-' * 80)
N_BOTH_SAME = len(DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == True) & (DF_RESULTS['disc_same_as_207'] == True)])
print(f'There are {N_BOTH_SAME:,} ({N_BOTH_SAME/len(DF_RESULTS):.2%})  hits where both halves have the same taxonomy')


There are 34,655 (99.45%) keep halves that are identical to the expected R207 taxonomy.
There are 191 (0.55%) keep halves that are different to the expected R207 taxonomy.
Of those, there are 154 where the chimeric half has the expected R207 taxonomy (we picked the wrong half?)
--------------------------------------------------------------------------------
There are 33,755 (96.87%) chimeric halves that are identical to the expected R207 taxonomy.
There are 1,091 (3.13%) chimeric halves that are different to the expected R207 taxonomy.
--------------------------------------------------------------------------------
There are 1,245 (3.57%) cases were either half has an incorrect taxonomic assignment
--------------------------------------------------------------------------------
There are 1,208 (3.47%) cases where one half was wrong, and the other half was correct.
There are 37 cases where neither half is correct.
-------------------------------------------------------------------------

In [5]:
CNT_KEEP_HIGHEST_AGREE_TAX = Counter(DF_RESULTS[DF_RESULTS['keep_same_as_207'] == False]['keep_rank_agree'])
print('The following ranks are the highest rank at which the keep set differ from the expected tax:')
print(CNT_KEEP_HIGHEST_AGREE_TAX)

print('-' * 80)

CNT_DISC_HIGHEST_AGREE_TAX = Counter(DF_RESULTS[DF_RESULTS['disc_same_as_207'] == False]['disc_rank_agree'])
print('The following ranks are the highest rank at which the chimeric set differ from the expected tax:')
print(CNT_DISC_HIGHEST_AGREE_TAX)

# The discard set seems to be more skewed towards not finding a match.

The following ranks are the highest rank at which the keep set differ from the expected tax:
Counter({'g': 158, nan: 30, 'f': 2, 'o': 1})
--------------------------------------------------------------------------------
The following ranks are the highest rank at which the chimeric set differ from the expected tax:
Counter({nan: 1025, 'g': 62, 'f': 3, 'c': 1})


In [6]:
# How many of the keep set had a chimeric half that yielded the correct taxonomy?
DF_KEEP_DIFF_BUT_DISC_SAME = DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == True)]
N_KEEP_DIFF_BUT_DISC_SAME = len(DF_KEEP_DIFF_BUT_DISC_SAME)
print(f'There were {N_KEEP_DIFF_BUT_DISC_SAME:,} hits where the keep half was wrong, but the chimeric half was correct')

DF_DISAGREE_TAX = DF_KEEP_DIFF_BUT_DISC_SAME[DF_KEEP_DIFF_BUT_DISC_SAME['keep_tax'].apply(lambda x: len(x) > 7 if str(x) != 'nan' else False)]
print(f'Of those, there were {len(DF_DISAGREE_TAX):,} cases where the keep tax actually had an assignment')

DF_KEEP_DIFF_BUT_DISC_SAME

There were 154 hits where the keep half was wrong, but the chimeric half was correct
Of those, there were 136 cases where the keep tax actually had an assignment


,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
190,GCA_000761495.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,g,NaN,GCF_014769185.1,NaN,96.6072,0.0000,0.8876,0.0000,sp_rep,no_ani,False,True,False
666,GCA_001509115.1,d__Bacteria;p__Thermotogota;c__Thermotogae;o__...,NaN,d__Bacteria;p__Thermotogota;c__Thermotogae;o__...,NaN,s,NaN,GCF_003228205.1,0.0000,95.5633,0.0000,0.6159,no_ani,sp_rep,False,True,False
677,GCA_001543395.1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,NaN,g,NaN,GCA_902786745.1,NaN,95.0615,0.0000,0.8320,0.0000,sp_rep,no_ani,False,True,False
771,GCA_001695755.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,NaN,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,NaN,s,NaN,GCF_010725885.1,0.0000,97.9201,0.0000,0.8510,no_ani,sp_rep,False,True,False
1283,GCA_002279705.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,g,NaN,GCF_001713455.1,NaN,95.0073,0.0000,0.9444,0.0000,sp_rep,no_ani,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31755,GCF_017311415.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,g,NaN,GCF_001975705.1,NaN,95.2958,0.0000,0.9197,0.0000,sp_rep,no_ani,False,True,False
34955,GCF_900509435.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_900978845.1,0.0000,95.5462,0.0000,0.7455,no_ani,sp_rep,False,True,False
35120,GCF_900559355.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_900978845.1,0.0000,95.3600,0.0000,0.7489,no_ani,sp_rep,False,True,False
35612,GCF_902845755.1,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,g,s,GCF_900478295.1,GCF_003637685.1,95.5641,95.3182,0.9704,0.8600,sp_rep,sp_rep,False,True,False


In [7]:
DF_NONE_CORRECT = DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) & (DF_RESULTS['disc_same_as_207'] == False)]
print(f'There were {len(DF_NONE_CORRECT):,} cases where neither half got the correct taxonomy')

DF_NONE_CORRECT

There were 37 cases where neither half got the correct taxonomy


,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
2619,GCA_002728285.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,False,False,True
4566,GCA_007096555.1,d__Bacteria;p__Cyanobacteria;c__Cyanobacteriia...,d__Bacteria;p__Cyanobacteria;c__Cyanobacteriia...,NaN,g,NaN,GCA_007095785.1,NaN,95.4508,0.0000,0.8245,0.0000,sp_rep,no_ani,False,False,False
4886,GCA_009493725.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__E...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,False,False,True
5753,GCA_011523095.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,g,g,GCA_002690995.1,GCA_002690995.1,95.4819,95.0449,0.8734,0.5718,sp_rep,sp_rep,False,False,True
5755,GCA_011523145.1,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_af,no_af,False,False,True
6340,GCA_013213925.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,False,False,True
7521,GCA_016707075.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_af,no_af,False,False,True
7915,GCA_017394825.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__C...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_af,no_af,False,False,True
8132,GCA_017465765.1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,False,False,True
8888,GCA_018056875.1,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,no_af,no_af,False,False,True


In [8]:
DF_TO_INVESTIGATE = DF_RESULTS[(DF_RESULTS['keep_same_as_207'] == False) | (DF_RESULTS['disc_same_as_207'] == False)]
DF_TO_INVESTIGATE

,gid,expected_tax,keep_tax,disc_tax,keep_rank_agree,disc_rank_agree,keep_sp_rep,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
2,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,s,NaN,GCF_003697165.2,NaN,96.5879,0.0,0.7680,0.0,sp_rep,no_af,True,False,False
38,GCA_000335755.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,NaN,s,NaN,GCF_000315525.1,NaN,96.0835,0.0,0.9547,0.0,sp_rep,no_ani,True,False,False
145,GCA_000504645.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,NaN,s,NaN,GCA_000378885.1,NaN,97.1670,0.0,0.8945,0.0,sp_rep,no_ani,True,False,False
150,GCA_000508605.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,NaN,s,NaN,GCF_000148585.2,NaN,95.2903,0.0,0.8664,0.0,sp_rep,no_ani,True,False,False
152,GCA_000508885.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,NaN,s,NaN,GCF_000239295.1,NaN,98.7098,0.0,0.9185,0.0,sp_rep,no_af,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35695,GCF_905219725.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,NaN,s,NaN,GCA_910574635.1,NaN,97.4894,0.0,0.9644,0.0,sp_rep,no_ani,True,False,False
35700,GCF_907163115.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,NaN,g,NaN,GCF_005844005.1,NaN,95.1355,0.0,0.9252,0.0,sp_rep,no_ani,False,True,False
35713,GCF_910574845.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,NaN,s,NaN,GCF_900113155.1,NaN,98.0498,0.0,0.8419,0.0,sp_rep,no_af,True,False,False
35718,GCF_910576245.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,NaN,s,NaN,GCF_900113155.1,NaN,97.9744,0.0,0.8292,0.0,sp_rep,no_af,True,False,False


In [9]:
import pandas as pd

def get_tax_changes():
    rows = list()
    for row in DF_TO_INVESTIGATE.itertuples():

        expected_tax_split = row.expected_tax.split(';')

        cur_row = {
            'gid': row.gid,
            'expected_rank': row.expected_tax,
            'keep_tax': row.keep_type,
            'chim_tax': row.disc_type
        }

        if len(str(row.keep_tax)) > 10:
            keep_tax_split = row.keep_tax.split(';')

            # find the highest rank at which there is a disagreement
            for i, (e, k) in enumerate(zip(expected_tax_split, keep_tax_split)):
                if e != k:
                    cur_row['keep_tax'] = ';'.join(keep_tax_split[i:])
                    break
            else:
                cur_row['keep_tax'] = 'same'


        if len(str(row.disc_tax)) > 10:
            disc_tax_split = row.disc_tax.split(';')

            # find the highest rank at which there is a disagreement
            for i, (e, k) in enumerate(zip(expected_tax_split, disc_tax_split)):
                if e != k:
                    cur_row['chim_tax'] = ';'.join(disc_tax_split[i:])
                    break
            else:
                cur_row['chim_tax'] = 'same'

        rows.append(cur_row)

    return pd.DataFrame(rows)

DF_TAX_CHANGES = get_tax_changes()

DF_TAX_CHANGES.to_csv('/tmp/sp_clustering_tax_changes.tsv', sep='\t')

DF_TAX_CHANGES

,gid,expected_rank,keep_tax,chim_tax
0,GCA_000155005.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,same,no_af
1,GCA_000335755.1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,same,no_ani
2,GCA_000504645.1,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,same,no_ani
3,GCA_000508605.1,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,same,no_ani
4,GCA_000508885.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,same,no_af
...,...,...,...,...
1240,GCF_905219725.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,same,no_ani
1241,GCF_907163115.1,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,s__Pseudomonas_A sp000765155,no_ani
1242,GCF_910574845.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,same,no_af
1243,GCF_910576245.1,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,same,no_af
